In [1]:
from time import sleep
from datetime import datetime
from typing import List
import os

from dotenv import load_dotenv

load_dotenv()
openai_api_key = os.environ.get("OPENAI_API_KEY")

In [2]:
from genworlds.worlds.concrete.community_chat_interface.world import ChatInterfaceWorld

RAGWorld = ChatInterfaceWorld(
    name="RAGWorld",
    description="""A world simulating the real-world scenario where users seek answers 
    based on processed content from various data sources.""",
    id="rag-world"
)

RAGWorld.launch()

INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:     Started server process [17188]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:7456 (Press CTRL+C to quit)
INFO:     ('127.0.0.1', 61931) - "WebSocket /ws" [accepted]
INFO:     connection open
INFO:websocket:Websocket connected
[rag-world Thread] Connected to world socket server ws://127.0.0.1:7456/ws
INFO:rag-world Thread:Connected to world socket server ws://127.0.0.1:7456/ws


In [3]:
from objects.qdrant_bucket import QdrantBucket

# Instantiate the QdrantBucket Object
qdrant_bucket = QdrantBucket(id="qdrant_bucket", path="./databases/vector_store.qdrant")

# Incorporate the QdrantBucket into the Simulation
RAGWorld.add_object(qdrant_bucket)

INFO:     ('127.0.0.1', 61932) - "WebSocket /ws" [accepted]
INFO:     connection open
INFO:websocket:Websocket connected
[qdrant_bucket Thread] Connected to world socket server ws://127.0.0.1:7456/ws
INFO:qdrant_bucket Thread:Connected to world socket server ws://127.0.0.1:7456/ws


In [4]:
from objects.local_storage_bucket import LocalStorageBucket

# Example usage:
local_storage = LocalStorageBucket(id="local_storage_bucket")

# Incorporate the Local Storage Bucket into the Simulation
RAGWorld.add_object(local_storage)

INFO:     ('127.0.0.1', 61933) - "WebSocket /ws" [accepted]
INFO:     connection open
INFO:websocket:Websocket connected
[local_storage_bucket Thread] Connected to world socket server ws://127.0.0.1:7456/ws
INFO:local_storage_bucket Thread:Connected to world socket server ws://127.0.0.1:7456/ws


In [5]:
from genworlds.utils.test_user import TestUser

# Create a Testing User
test_user = TestUser()

INFO:     ('127.0.0.1', 61934) - "WebSocket /ws" [accepted]
INFO:     connection open
INFO:websocket:Websocket connected
[test_user Thread] Connected to world socket server ws://127.0.0.1:7456/ws
INFO:test_user Thread:Connected to world socket server ws://127.0.0.1:7456/ws


In [6]:
from genworlds.agents.concrete.basic_assistant.utils import generate_basic_assistant
from genworlds.worlds.concrete.base.actions import UserSpeaksWithAgentEvent
from objects.qdrant_bucket import VectorStoreCollectionCreated


agent_name = "DCPI"
description = """Agent that helps the user process files and create data structures 
such as vector stores using available objects."""

# Generate a Dummy Agent named DCPI
dcpi = generate_basic_assistant(
    agent_name=agent_name, 
    description=description,
    openai_api_key=openai_api_key
)

# We add the vector_store_collection_created event to the wakeup_events dictionary because it can take several minutes to complete
dcpi.add_wakeup_event(event_class=UserSpeaksWithAgentEvent)
dcpi.add_wakeup_event(event_class=VectorStoreCollectionCreated)

## Attach DCPI to the Simulation
RAGWorld.add_agent(dcpi)

INFO:     ('127.0.0.1', 61935) - "WebSocket /ws" [accepted]
INFO:     connection open
INFO:websocket:Websocket connected
[DCPI Thread] Connected to world socket server ws://127.0.0.1:7456/ws
INFO:DCPI Thread:Connected to world socket server ws://127.0.0.1:7456/ws


{"event_type": "agent_wants_updated_state", "description": "Agent wants to update its state.", "summary": null, "created_at": "2023-10-02T22:40:35.353476", "sender_id": "DCPI", "target_id": "rag-world"}
{"event_type": "world_sends_available_entities_event", "description": "Send available entities.", "summary": null, "created_at": "2023-10-02T22:40:35.356543", "sender_id": "rag-world", "target_id": "DCPI", "available_entities": {"rag-world": {"id": "rag-world", "entity_type": "WORLD", "entity_class": "ChatInterfaceWorld", "name": "RAGWorld", "description": "A world simulating the real-world scenario where users seek answers \n    based on processed content from various data sources."}, "DCPI": {"id": "DCPI", "entity_type": "AGENT", "entity_class": "BasicAssistant", "name": "DCPI", "description": "Agent that helps the user process files and create data structures \nsuch as vector stores using available objects."}, "qdrant_bucket": {"id": "qdrant_bucket", "entity_type": "OBJECT", "entity_



> Entering new LLMChain chain...
Prompt after formatting:
System: You are DCPI, Agent that helps the user process files and create data structures 
such as vector stores using available objects..

System: You are embedded in a simulated world with those properties 

System: Those are your goals: 
['Starts waiting and sleeps till the user starts a new question.', "Once DCPI receives a user's question, he makes sure to have all the information before sending the answer to the user.", 'When DCPI has all the required information, he speaks to the user with the results through the agent_speaks_with_user_event.', 'After sending the response, he waits for the next user question.', 'If you have been waiting for any object or entity to send you an event for over 30 seconds, you will wait and sleep until you receive a new event.']

System: And this is your current plan to achieve the goals: 
[]

System: Here is your memories of all the events that you remember from being in this simulation: 



In [7]:
from genworlds.worlds.concrete.base.actions import UserSpeaksWithAgentEvent

# Format the message that will be sent to the simulation socket
test_msg = """DCPI Agent, please convert the documents located './databases/genworlds-docs' 
into a single txt and store it as ./databases/genworlds-docs.txt """

message_to_send = UserSpeaksWithAgentEvent(
    sender_id=test_user.id, 
    created_at=datetime.now(),
    message=test_msg, 
    target_id="DCPI"
).json()

sleep(1)

# Send the message to DCPI
test_user.socket_client.send_message(message_to_send)

In [8]:
from genworlds.worlds.concrete.base.actions import UserSpeaksWithAgentEvent

# Format the message that will be sent to the simulation socket
test_msg = """DCPI Agent, please create a collection named genworlds-text-chunks in the qdrant vectorstore
from the contents of this file ./databases/genworlds-docs.txt """

message_to_send = UserSpeaksWithAgentEvent(
    sender_id=test_user.id, 
    created_at=datetime.now(),
    message=test_msg, 
    target_id="DCPI"
).json()

sleep(1)

# Send the message to DCPI
test_user.socket_client.send_message(message_to_send)

In [9]:
from genworlds.worlds.concrete.base.actions import UserSpeaksWithAgentEvent

test_msg = """DCPI Agent, please create a collection named genworlds-ner which extracts Named Entities and its descriptions in the qdrant vectorstore
from the contents of this file ./databases/genworlds-docs.txt """

message_to_send = UserSpeaksWithAgentEvent(
    sender_id=test_user.id, 
    created_at=datetime.now(),
    message=test_msg, 
    target_id="DCPI"
).json()

sleep(1)

# Send the message to DCPI
test_user.socket_client.send_message(message_to_send)

In [10]:
from genworlds.worlds.concrete.base.actions import UserSpeaksWithAgentEvent

test_msg = """DCPI Agent, please create a collection named genworlds-text-chunks in the qdrant vectorstore
from the contents of this file ./databases/genworlds-docs.txt """

message_to_send = UserSpeaksWithAgentEvent(
    sender_id=test_user.id, 
    created_at=datetime.now(),
    message=test_msg, 
    target_id="DCPI"
).json()

sleep(1)

# Send the message to DCPI
test_user.socket_client.send_message(message_to_send)

INFO:     ('127.0.0.1', 61988) - "WebSocket /ws" [accepted]
INFO:     connection open
INFO:     ('127.0.0.1', 62003) - "WebSocket /ws" [accepted]
INFO:     connection open
